レース情報をスクレイピング

jupyter notebook で [ImportError: lxml not found, please install it]エラーが出た場合は [pip3 install lxml] でインストール後プロジェクトを再起動


In [13]:
import pandas as pd
from tqdm import tqdm
import time

url >> race_id=2022 09 01 11 01 : 年  競馬場(01:札幌,02:函館,03:福島,04:新潟,05:東京,06:中山,07:中京,08:京都,09:阪神,10:小倉) 何回(今年その競馬場でのレース回数) 何日目(何レース目)

In [14]:
pd.set_option('display.max_rows',10)
pd.options.display.width = 200
pd.options.display.column_space = 100

スクレイピング

In [15]:
def scrape_race_results(race_id_lists,par_race_results={}):
  race_result_dict = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      race_result_dict[_race_id] = pd.read_html(url)[0]
      # print(race_result_dict[race_id])
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return race_result_dict

# race_result = {}

dictのkeyをrace_idに変換


In [16]:
def get_house_data():
  race_id_lists = []

  for 年 in range(2020,2021):
    for 競馬場 in range(1,3): # 1,11
      for 回 in range(1,11): # 1,11
        for 回目 in range(1,11): # 1,11
          for レース回 in range(1,13): # 1,13
            race_id = f"{年}{str(競馬場).zfill(2)}{str(回).zfill(2)}{str(回目).zfill(2)}{str(レース回).zfill(2)}"
            race_id_lists.append(race_id)
            
  race_result = scrape_race_results(race_id_lists)
  for key in race_result.keys():
    race_result[key].index = [key] * len(race_result[key])
    
  results = pd.concat([race_result[key] for key in race_result.keys()],sort=False)
  return results

新規でdata取得するとき

In [17]:
house_data = get_house_data()

100%|██████████| 2400/2400 [59:08<00:00,  1.48s/it]   


追加でdata取得するとき

In [18]:
len(house_data)

3761

In [19]:
house_data.to_pickle('results.pickle')

In [20]:
# pd.read_pickle('results.pickle')